In [ ]:
import os
os.chdir(r'C:\Users\xxcmt\Desktop\文档\automate_online-materials')
fileObj=open('hello.txt','w')
fileObj.write('Hello world!')
fileObj.close()
 fileObj.close()
    
import subprocess
subprocess.Popen(['start','hello.txt'],shell=True)


In [ ]:
import time,datetime，requests
delta=datetime.timedelta(days=1)
dt=datetime.datetime(2019,5,12,8)
print(dt)
while True:
    if delta.total_seconds()>0:
        print('还没到运行的时候')
        halfdaydelta=datetime.timedelta(hours=12)
        delta=delta-halfdaydelta
        time.sleep(halfdaydelta)
    else:
        #运行查看漫画程序
        
        delta=datetime.timedelta(days=1)
    
    
    
    



In [ ]:
import time,subprocess,os
os.chdir(r'C:\Users\xxcmt\Desktop\文档\automate_online-materials')
timeLeft=60
while timeLeft>0:
    print(timeLeft,end='')
    time.sleep(1)
    timeLeft=timeLeft-1
subprocess.Popen(['start',r'C:\Users\xxcmt\Desktop\文档\automate_online-materials\alarm.wav'],shell=True)

In [6]:
import os,json
os.chdir(r'C:/Users/xxcmt/Desktop/文档/automate_online-materials/comicPictures/')
dictcomic={}
with open(r'log.json','w') as f:
    json.dump(dictcomic,f,ensure_ascii=False)
    print("清空日志...")
'''
with open(r'log.json','r') as load_f:
    print(type(load_f))
    load_dict=json.load(load_f)
    print(load_dict)
#    print(load_dict)
#    print(load_f.read())
'''



加载入文件完成...


"\nwith open(r'log.json','r') as load_f:\n    print(type(load_f))\n    load_dict=json.load(load_f)\n    print(load_dict)\n#    print(load_dict)\n#    print(load_f.read())\n"

In [9]:
import datetime,time,sys,subprocess
delta=datetime.timedelta(days=1)
oneday=delta.total_seconds()
while oneday>0:
    time.sleep(1)
    oneday=oneday-1
    subprocess.Popen([r'C:\Users\xxcmt\Anaconda3\python.exe',r'C:\Users\xxcmt\Downloads\downLoadComic.py'],shell=True)
    


KeyboardInterrupt: 

In [5]:
#爬虫有学问，骚年还得多看书!!!!!!!!!!!!!!!!
from selenium import webdriver
import time,pprint,bs4,os
from selenium.webdriver.common.keys import Keys
import requests,threading
import json,sys

def getLatesturl():
    os.chdir(r'C:\Users\xxcmt\Desktop\文档\automate_online-materials')
    res=requests.get('https://ac.qq.com/Comic/comicInfo/id/525099')
    soup=bs4.BeautifulSoup(res.text)
    #检查是否为最新html 
    capterElem=soup.select('#chapter > div.works-chapter-top.subscribe-wrap > ul > li:nth-child(2) > a')
    latestCapterName=capterElem[0].get_text()
    with open(r'.\comicPictures\log.json','r') as load_f:
        latestCapter_dict=json.load(load_f)
    print('打开日志检查是否为最新章')
    latestCapterName='恶偶'+latestCapterName#完善日志的key
    if latestCapterName not in latestCapter_dict.keys():
        latestComicurl='https://ac.qq.com/'+capterElem[0].get('href')
        latestCapter_dict[latestCapterName]=latestComicurl
        #将最新章的日志写入
        with open(r'.\comicPictures\log.json','w') as f:
            json.dump(latestCapter_dict,f,ensure_ascii=False)
        eouFloder=os.path.join(r'C:\Users\xxcmt\Desktop\文档\automate_online-materials\comicPictures','恶偶',latestCapterName)
        os.makedirs(eouFloder,exist_ok=True)
        print("检测到最新章为%s，开始下载..."%latestCapterName)
        return latestComicurl,eouFloder
    else:
        print('已经为最新章节')
        return -1,-1
   
lastUrl,picture_Floder=getLatesturl()
if lastUrl!=-1:
    browser = webdriver.Firefox()
    browser.get(lastUrl)
    time.sleep(2)
    htmlElem = browser.find_element_by_tag_name('html')
else:
    os._exit(0)
    
#模拟pressDown的动作，并更新源码
def pressPageDown():
    htmlElem.send_keys(Keys.PAGE_DOWN)
    htmlElem.send_keys(Keys.PAGE_DOWN)
    htmlElem.send_keys(Keys.PAGE_DOWN)
    htmlElem.send_keys(Keys.PAGE_DOWN) 
    time.sleep(2)
    html_text=browser.page_source
    os.chdir(r'C:\Users\xxcmt\Desktop\文档\automate_online-materials')
    with open('badidol.html','w',encoding='utf-8') as f:
        f.write(html_text)
            
#读取源码，检查是否下载完
increasePageNum={}
def isCompleteDownLoad():
    with open('badidol.html','r',encoding='utf-8') as f:
        soup=bs4.BeautifulSoup(f,'html')
    imgElems=soup.select('#comicContain  li  img')
    for i in imgElems:
        if i.get('src').startswith('https://manhua.qpic.cn/manhua_detail') is not True:
            imgElems.remove(i)
    print('已加载%s张图片'%len(imgElems))
    #因为缓存的原因，所以容错机制连续4次press没有变化的情况，表示加载完毕
    if len(imgElems) not in increasePageNum.keys():
        increasePageNum[len(imgElems)]=0
        return 0,imgElems
    else:
        increasePageNum[len(imgElems)]=increasePageNum[len(imgElems)]+1
        if increasePageNum[len(imgElems)]==3:
            return 1,imgElems
        else:
            return 0,imgElems
           
#先pressdown一次，再检查，如此往覆,返回完整的imgElems
def downloadLazyComic():
    pressPageDown()
    isComplete,imgElems=isCompleteDownLoad()
    while isComplete==0:
        pressPageDown()
        isComplete,imgElems=isCompleteDownLoad()
    return imgElems

#多线程下载
imgElems=downloadLazyComic()
browser.quit()

os.chdir(picture_Floder)
#分成5或者6个线程
i=5
while True:
    step=len(imgElems)//i
    if step==0:
        i=i-1
    else:
        break
        
#下载从a到b的图像
def downloadComic(a,b):
    for i in range (a,b):
        url=imgElems[i].get('src')
        res=requests.get(url)
        imageFile=open(str(i+1)+'.jpg','wb') 
        for chunk in res.iter_content(100000):
            imageFile.write(chunk)
        imageFile.close()
        print('picture %s has downloaded'%(i+1))

downloadThreads=[]
for i in range(0,len(imgElems),step):
    downloadThread=threading.Thread(target=downloadComic,args=(i,min(i+step,len(imgElems))))
    downloadThreads.append(downloadThread)
    downloadThread.start()
for downloadThread in downloadThreads:
    downloadThread.join()
print('done')



    



打开日志检查是否为最新章
检测到最新章为恶偶[公告-休更2周]，开始下载...
1
1
1
1
picture 0 has downloaded
done


In [ ]:
#写一个判断是否下载完整的fuc，输入为html_text，返回为true,false,参数部分为list,imgElems,
import os,requests,bs4
os.chdir(r'C:\Users\xxcmt\Desktop\文档\automate_online-materials')
#放图像的list
def isCompleteDownLoad():
    with open('badidol.html','r',encoding='utf-8') as f:
        soup=bs4.BeautifulSoup(f,'html')
    imgElems=soup.select('#comicContain  li  img')
    for i in imgElems:
        if i.get('src').startswith('https://manhua.qpic.cn/manhua_detail') is not True:
            imgElems.remove(i)
    pageElems=soup.select('#comicContain li div em')
    print(len(imgElems))
    return (len(pageElems)==len(imgElems),imgElems)
isCompleteDownLoad,imgElems=isCompleteDownLoad()
if isCompleteDownLoad:
    print('OK')
print(len(imgElems))

    

In [ ]:
hello=['hello','world']
def printhello(hello):
    hello.remove('hello')
    print(hello[0])
printhello(hello)
print(hello[0])